# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# load messages dataset
messages = pd.read_csv('disaster_messages.csv')
origNum = messages.shape[0]
print(origNum)

In [ ]:
# load categories dataset
categories = pd.read_csv('disaster_categories.csv')
categories.shape[0]

In [ ]:
(messages['id'] != categories['id']).sum()

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [ ]:
# merge datasets
df = pd.merge(left=messages, right=categories, left_on='id', right_on='id', how='inner')
df.shape[0]

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [ ]:
# create a dataframe of the 36 individual category columns

# The 'id' column will be lost in the split.  Let's save a copy.
idSeries = categories['id']
idSeries.name='id'

categories = categories.categories.str.split(';', expand=True)
categories.shape

In [ ]:
# select the first row of the categories dataframe
row = categories.iloc[0]
category_colnames = list(row.str.split('-', expand=True)[0])

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
print(category_colnames)

In [ ]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.shape

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [ ]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str.slice(start=-1).astype(int)
    
    # convert column from string to numeric
    # categories[column] = <too late, already did it.>

In [ ]:
# correct non-binary values in `related` column, 2 -> 0
categories['related'].replace(to_replace=[2], value=0, inplace=True)

### 4b. Restore the 'id' column to `categories`

In [ ]:
# DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, 
# sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

categories = pd.merge(left=pd.DataFrame(idSeries), right=categories, left_index=True, right_index=True)

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
# drop the "original" categories column from `df`
df.drop('categories', axis=1, inplace=True)

In [ ]:
df = pd.merge(left=df, right=categories, left_on='id', right_on='id')

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
lastNum = df.shape[0]
print(lastNum)

In [ ]:
# check number of duplicates
# df.groupby(by='message').size().sort_values(ascending=False)
pivot = df.pivot_table(index=['message'], aggfunc='size').sort_values(ascending=False)
# Turns out there's an easier way to do this... something about df.duplicate.  I should check it out.

In [ ]:
# drop duplicates
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
# check number of duplicates
lastNum - df.shape[0]

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [ ]:
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('MessageCategorization', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

Now called `process_data.py`.